In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import precision_recall_curve, average_precision_score
from ultralytics import YOLO

In [ ]:
# Function to load models
def load_models(model_paths):
    """Load YOLOv8 models from given paths."""
    models = {os.path.basename(path): YOLO(path) for path in model_paths}
    return models

In [ ]:
# Function to perform inference
def perform_inference(model, dataset_path):
    """Run inference on a dataset and return predictions."""
    predictions = []
    images = []
    for image_file in os.listdir(dataset_path):
        image_path = os.path.join(dataset_path, image_file)
        image = cv2.imread(image_path)
        results = model(image)
        predictions.append(results)
        images.append(image)
    return predictions, images

In [ ]:
# Function to evaluate metrics
def evaluate_metrics(predictions, ground_truths, class_names):
    """Evaluate precision, recall, and mAP per class."""
    metrics = {cls: {"precision": [], "recall": [], "AP": 0.0} for cls in class_names}

    for cls_idx, cls_name in enumerate(class_names):
        y_true = []
        y_scores = []
        for pred, gt in zip(predictions, ground_truths):
            gt_cls = [1 if obj["class"] == cls_idx else 0 for obj in gt]
            pred_cls = [d.conf for d in pred if d.cls == cls_idx]

            y_true.extend(gt_cls)
            y_scores.extend(pred_cls)

        precision, recall, _ = precision_recall_curve(y_true, y_scores)
        ap = average_precision_score(y_true, y_scores)

        metrics[cls_name]["precision"] = precision
        metrics[cls_name]["recall"] = recall
        metrics[cls_name]["AP"] = ap

    return metrics

In [ ]:
# Function to visualize metrics
def visualize_metrics(metrics):
    """Visualize precision-recall curves per class."""
    for cls, values in metrics.items():
        plt.figure()
        plt.plot(values["recall"], values["precision"], label=f'{cls} (AP={values["AP"]:.2f})')
        plt.xlabel("Recall")
        plt.ylabel("Precision")
        plt.title(f"Precision-Recall Curve for {cls}")
        plt.legend()
        plt.grid()
        plt.show()

In [ ]:
# Function to display results summary
def display_summary(metrics):
    """Print and summarize evaluation results."""
    print("\nEvaluation Summary\n------------------")
    for cls, values in metrics.items():
        print(f"Class: {cls}")
        print(f"  AP: {values['AP']:.2f}")

In [ ]:
# Main function to run evaluation
def main(model_paths, dataset_path, ground_truths, class_names):
    models = load_models(model_paths)

    for model_name, model in models.items():
        print(f"Evaluating Model: {model_name}")

        predictions, images = perform_inference(model, dataset_path)
        metrics = evaluate_metrics(predictions, ground_truths, class_names)

        visualize_metrics(metrics)
        display_summary(metrics)